In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('student-por.csv')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [3]:
df_dropped = df[['sex', 'age', 'Medu', 'reason', 'traveltime', 'studytime', 'freetime', 'higher',
       'failures', 'internet', 'G3']]
df_dropped

,sex,age,Medu,reason,traveltime,studytime,freetime,higher,failures,internet,G3
0,F,18,4,course,2,2,3,yes,0,no,11
1,F,17,1,course,1,2,3,yes,0,yes,11
2,F,15,1,other,1,2,3,yes,0,yes,12
3,F,15,4,home,1,3,2,yes,0,yes,14
4,F,16,3,home,1,2,3,yes,0,no,13
...,...,...,...,...,...,...,...,...,...,...,...
644,F,19,2,course,1,3,4,yes,1,yes,10
645,F,18,3,course,1,2,3,yes,0,yes,16
646,F,18,1,course,2,2,1,yes,0,no,9
647,M,17,3,course,2,1,4,yes,0,yes,10


In [43]:
#Numericals to scale
df_toscale = df_dropped.select_dtypes(exclude='object')
df_toscale

,age,Medu,traveltime,studytime,freetime,failures,G3
0,18,4,2,2,3,0,11
1,17,1,1,2,3,0,11
2,15,1,1,2,3,0,12
3,15,4,1,3,2,0,14
4,16,3,1,2,3,0,13
...,...,...,...,...,...,...,...
644,19,2,1,3,4,1,10
645,18,3,1,2,3,0,16
646,18,1,2,2,1,0,9
647,17,3,2,1,4,0,10


In [46]:
#Categorical variables 
df_toencode = df_dropped.select_dtypes(include='object')
df_toencode

,sex,reason,higher,internet
0,F,course,yes,no
1,F,course,yes,yes
2,F,other,yes,yes
3,F,home,yes,yes
4,F,home,yes,no
...,...,...,...,...
644,F,course,yes,yes
645,F,course,yes,yes
646,F,course,yes,no
647,M,course,yes,yes


In [37]:
#Pre-processing pipeline
# Build the pipeline with the different steps
pipeline = Pipeline([
    ('encoder', OneHotEncoder(drop="if_binary")),
   ])

pipeline



Pipeline(steps=[('encoder', OneHotEncoder(drop='if_binary'))])

In [31]:
pipeline.fit(df_dropped[['Medu']])
failure_transformed = pipeline.transform(df_dropped[['Medu']])
df_dropped

,sex,age,Medu,reason,traveltime,studytime,freetime,higher,failures,internet,G3
0,F,18,4,course,2,2,3,yes,0,no,11
1,F,17,1,course,1,2,3,yes,0,yes,11
2,F,15,1,other,1,2,3,yes,0,yes,12
3,F,15,4,home,1,3,2,yes,0,yes,14
4,F,16,3,home,1,2,3,yes,0,no,13
...,...,...,...,...,...,...,...,...,...,...,...
644,F,19,2,course,1,3,4,yes,1,yes,10
645,F,18,3,course,1,2,3,yes,0,yes,16
646,F,18,1,course,2,2,1,yes,0,no,9
647,M,17,3,course,2,1,4,yes,0,yes,10
